In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [8]:
df1 = df1.apply(pd.to_numeric, errors = 'coerce')
df1.describe().iloc[:, 13:]
# 없는 Column = date, codesum, depart, sunrise, sunset, station_nbr, snowfall, 

,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,102564.000000,102786.00000,102231.000000,102675.000000,102675.000000,102786.000000,103119.000000,103119.000000,103119.000000,103119.0,103119.0,103119.000000,103119.000000,103119.00000,103119.000000
mean,0.075666,29.85054,30.016037,5.316649,20.257297,6.528402,0.493003,0.500538,0.079656,1.0,1.0,56.000000,0.364909,3.00000,0.285253
std,0.191589,0.22727,0.227675,3.378105,9.823834,3.247471,0.499953,0.500002,0.270761,0.0,0.0,32.041795,3.609916,2.00001,0.451537
min,0.000000,28.95000,29.200000,0.200000,1.000000,0.600000,0.000000,0.000000,0.000000,1.0,1.0,1.000000,0.000000,0.00000,0.000000
25%,0.000000,29.70000,29.870000,2.700000,11.000000,4.200000,0.000000,0.000000,0.000000,1.0,1.0,28.000000,0.000000,1.00000,0.000000
50%,0.005000,29.85000,30.020000,4.700000,22.000000,5.900000,0.000000,1.000000,0.000000,1.0,1.0,56.000000,0.000000,3.00000,0.000000
75%,0.030000,30.01000,30.170000,7.400000,28.000000,8.200000,1.000000,1.000000,0.000000,1.0,1.0,84.000000,0.000000,5.00000,1.000000
max,1.980000,30.59000,30.770000,17.900000,36.000000,19.800000,1.000000,1.000000,1.000000,1.0,1.0,111.000000,207.000000,6.00000,1.000000


In [9]:
df1.describe()

,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,...,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,1.031190e+05,102897.000000,102897.000000,102564.000000,0.0,102675.000000,102342.000000,102564.000000,102564.000000,0.0,...,102786.000000,103119.000000,103119.000000,103119.000000,103119.0,103119.0,103119.000000,103119.000000,103119.00000,103119.000000
mean,1.367872e+18,60.641855,41.666667,51.427489,NaN,37.944865,45.251627,15.636364,2.063853,NaN,...,6.528402,0.493003,0.500538,0.079656,1.0,1.0,56.000000,0.364909,3.00000,0.285253
std,2.591248e+16,18.975950,16.784905,17.591533,NaN,19.044399,16.126532,15.041496,4.321170,NaN,...,3.247471,0.499953,0.500002,0.270761,0.0,0.0,32.041795,3.609916,2.00001,0.451537
min,1.325376e+18,14.000000,-4.000000,8.000000,NaN,-12.000000,4.000000,0.000000,0.000000,NaN,...,0.600000,0.000000,0.000000,0.000000,1.0,1.0,1.000000,0.000000,0.00000,0.000000
25%,1.345421e+18,45.000000,29.000000,38.000000,NaN,23.000000,33.000000,0.000000,0.000000,NaN,...,4.200000,0.000000,0.000000,0.000000,1.0,1.0,28.000000,0.000000,1.00000,0.000000
50%,1.365552e+18,62.000000,42.000000,53.000000,NaN,39.000000,46.000000,12.000000,0.000000,NaN,...,5.900000,0.000000,1.000000,0.000000,1.0,1.0,56.000000,0.000000,3.00000,0.000000
75%,1.391040e+18,77.000000,56.000000,66.000000,NaN,54.000000,59.000000,27.000000,1.000000,NaN,...,8.200000,1.000000,1.000000,0.000000,1.0,1.0,84.000000,0.000000,5.00000,1.000000
max,1.414714e+18,97.000000,77.000000,86.000000,NaN,71.000000,76.000000,57.000000,21.000000,NaN,...,19.800000,1.000000,1.000000,1.000000,1.0,1.0,111.000000,207.000000,6.00000,1.000000


In [3]:
df13['store_nbr'].unique()

array([15, 25, 32, 37, 40], dtype=int64)

In [6]:
df13_drop_columns = ['date', 'codesum', 'depart', 'sunrise', 'sunset', 'station_nbr', 'snowfall']
df13 = df13.drop(columns = df13_drop_columns)

In [7]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df13_columns = df13.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df13_columns:
    if (i == 'resultdir'):
        df13[i].fillna(df13[i].mode()[0], inplace=True)
        print(df13[i].mode()[0])
    else:
        df13[i].fillna(df13[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df13['relative_humility'] = 100*(np.exp((17.625*((df13['dewpoint']-32)/1.8))/(243.04+((df13['dewpoint']-32)/1.8)))/np.exp((17.625*((df13['tavg']-32)/1.8))/(243.04+((df13['tavg']-32)/1.8))))

# 체감온도 계산
df13["windchill"] = 35.74 + 0.6215*df13["tavg"] - 35.75*(df13["avgspeed"]**0.16) + 0.4275*df13["tavg"]*(df13["avgspeed"]**0.16)

df13 = df13[df13['units'] != 0]

16


In [8]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df13 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df13)
result_df13 = model_df13.fit()

print(result_df13.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.870
Method:                 Least Squares   F-statistic:                     2442.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:28:57   Log-Likelihood:                -11474.
No. Observations:               15369   AIC:                         2.303e+04
Df Residuals:                   15326   BIC:                         2.336e+04
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]          0.0235      0

In [9]:
model_df13 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df13)
result_df13 = model_df13.fit()

print(result_df13.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.870
Method:                 Least Squares   F-statistic:                     2442.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:29:12   Log-Likelihood:                -11474.
No. Observations:               15369   AIC:                         2.303e+04
Df Residuals:                   15326   BIC:                         2.336e+04
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [10]:
anova_result_df13 = sm.stats.anova_lm(result_df13).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df13[anova_result_df13['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(resultspeed),1.0,2.088198,2.088198,7.989989,4.709759e-03
C(rain),1.0,2.816703,2.816703,10.777438,1.029748e-03
scale(preciptotal),1.0,2.879768,2.879768,11.018740,9.040573e-04
C(other),1.0,3.735580,3.735580,14.293303,1.570137e-04
scale(sealevel),1.0,4.951022,4.951022,18.943898,1.354910e-05
scale(heat),1.0,9.652312,9.652312,36.932257,1.252018e-09
scale(tmax),1.0,9.972551,9.972551,38.157577,6.690386e-10
C(week7),6.0,319.901395,53.316899,204.004335,3.119668e-251
C(item_nbr),21.0,26446.551292,1259.359585,4818.637580,0.000000e+00
C(nothing),2.0,144920.513412,72460.256706,277251.803302,0.000000e+00


In [11]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df13.values, i) for i in range(df13.shape[1])]
vif["features"] = df13.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# resultspeed, preciptotal, other, week7, item_nb

,VIF Factor,features
0,1.012089,store_nbr
1,1.045838,item_nbr
2,1.054368,units
3,1.125243,resultdir
4,1.315151,preciptotal
5,1.714148,other
6,2.772391,week7
7,2.787085,weekend
8,3.670228,resultspeed
9,7.144415,avgspeed


In [12]:
# resultspeed, preciptotal, other, week7, item_nb
model_df13 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(other) + C(week7)+ scale(preciptotal) + \
+ scale(resultspeed) + 0', data = df13)
result_df13 = model_df13.fit()

print(result_df13.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.869
Model:                            OLS   Adj. R-squared:                  0.869
Method:                 Least Squares   F-statistic:                     3394.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:31:10   Log-Likelihood:                -11530.
No. Observations:               15369   AIC:                         2.312e+04
Df Residuals:                   15338   BIC:                         2.336e+04
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(item_nbr)[5]         3.8186      0

In [13]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
X13 = df13[['item_nbr', 'other', 'week7', 'preciptotal', 'resultspeed']]
y13 = df13['units']
model13 = LinearRegression()

cv13 = KFold(n_splits=13, shuffle=True, random_state=0)

cross_val_score(model13, X13, y13, scoring="r2", cv=cv13)

array([0.09684695, 0.09322251, 0.1044634 , 0.10758238, 0.09287561,
       0.08017   , 0.09520012, 0.01905147, 0.07604049, 0.07482086,
       0.09209159, 0.08413179, 0.0086498 ])